# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df = pd.read_csv('weather_data.csv')
weather_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness %,Wind Speed (mph)
0,0,severo-kurilsk,50.6789,156.1250,39.830,79,19,2.87
1,1,wloszczowa,50.8526,19.9659,52.934,79,100,0.89
2,2,mataura,-46.1927,168.8643,55.274,70,62,6.71
3,3,airai,-8.9266,125.4092,73.472,60,18,1.56
4,4,salalah,17.0151,54.0924,82.670,83,75,1.54
...,...,...,...,...,...,...,...,...
1306,1306,yangjiang,21.8500,111.9667,80.276,56,92,0.50
1307,1307,kinablangan,7.6947,126.5503,83.876,68,100,1.97
1308,1308,kropotkin,45.4375,40.5756,52.610,95,76,3.43
1309,1309,thunder bay,48.4001,-89.3168,45.932,61,75,5.66


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=g_key)

In [17]:
locations = weather_df[["Latitude", "Longitude"]]
humid = weather_df['Humidity']
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [65]:
#temp below 80 
#100% cloudy
#wind speed over 10mph

hotel_df = weather_df[weather_df['Temperature'] < 80]
hotel_df = hotel_df[hotel_df['Cloudiness %'] == 100]
hotel_df = hotel_df[hotel_df['Wind Speed (mph)'] > 10]
hotel_df['Hotel Name'] = ""
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness %,Wind Speed (mph),Hotel Name
340,340,berlevag,70.8578,29.0864,29.570,68,100,11.79,
699,699,skjervoy,70.0311,20.9714,30.776,62,100,10.59,
1253,1253,honningsvag,70.9821,25.9704,29.984,56,100,10.49,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [66]:
# 4. Find a balloon store near the White House.
hotel_names = []
hotels_lat = []
hotels_lng = []
for i, row in hotel_df.iterrows():
    search = "Hotel"
    coords = f"{row['Latitude']},{row['Longitude']}"
    radius = 5000
    
    # redefine params
    params = {
        "location": coords,
        "keyword": search,
        "radius": radius,
        "key": g_key
    }

    # Build URL using the Google Maps API
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Run request
    current_hotel = requests.get(base_url, params).json()
   
    hotel_name = current_hotel["results"][0]["name"]
    hotel_lat = current_hotel["results"][0]["geometry"]["location"]["lat"]
    hotel_lng = current_hotel["results"][0]["geometry"]["location"]["lng"]

    hotel_names.append(hotel_name)
    hotels_lat.append(hotel_lat)
    hotels_lng.append(hotel_lng)

hotel_df['Hotel Names'] = hotel_names
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness %,Wind Speed (mph),Hotel Name,Hotel Names
340,340,berlevag,70.8578,29.0864,29.570,68,100,11.79,,Berlevåg camping & Appartement AS
699,699,skjervoy,70.0311,20.9714,30.776,62,100,10.59,,Hotel Maritim Skjervøy AS
1253,1253,honningsvag,70.9821,25.9704,29.984,56,100,10.49,,Scandic Bryggen


In [75]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Country'

In [74]:
# Add marker layer ontop of heat map
coordinates = zip(hotels_lat, hotels_lng)
coordinates = list(coordinates)
# Display figure
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))